# Real Quantum Amplitude Estimation (RQAE) module

The present notebook reviews the **Real Quantum Amplitude Estimation** (RQAE) algorithms which were implemented into the module *real_quantum_ae* within the package *AE* of the library *QQuantLib* (**QQuantLib/AE/real_quantum_ae.py**).

$$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$$
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

The present notebook and module are based on the following references:

* Manzano, A., Musso, D. & Leitao, Á. Real quantum amplitude estimation. EPJ Quantum Technol. 10, 2 (2023) (https://epjquantumtechnology.springeropen.com/articles/10.1140/epjqt/s40507-023-00159-0#citeas)


In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver. QPU = [qlmass, python, c]
from QQuantLib.qpu.get_qpu import get_qpu
QPU = ["qlmass", "python", "c"]
linalg_qpu = get_qpu(QPU[2])

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results
from QQuantLib.utils.utils import bitfield_to_int

## 1. Oracle generation

Before doing any amplitude estimation we want to load some data into the quantum circuit, as this step is only auxiliary to see how the algorithm works, we are just going to load a discrete probability distribution. In this case, we will have a circuit with $n=3$ qubits which makes a total of $N = 2^n = 8$ states. The discrete probability distribution that we are going to load is:
$$p_d = \dfrac{(0,1,2,3,4,5,6,7)}{0+1+2+3+4+5+6+7+8}.$$


In [ ]:
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)

Note that this probability distribution is properly normalised. For loading this probability into the quantum circuit we will use the function *load_probability* from **QQuantLib/DL/data_loading** module. The state that we are going to get is:
    $$|\Psi\rangle = \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

In [ ]:
from QQuantLib.DL.data_loading import load_probability

In [ ]:
oracle = load_probability(probability)

For more information about loading data into the quantum circuit see the notebook *01_DataLoading_Module_Use*.

## 2. RQAE class

### 2.1 The Amplitude Estimation Problem

The **RQAE** algorithm solves the **amplitude estimation** problem when a little variation is added.  In this case, given an oracle:

$$\mathcal{0}|0\rangle = |\Psi\rangle = a|\Psi_0\rangle +\sqrt{1-a^2}|\Psi_1\rangle, \tag{1}$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, *we want to estimate the real parameter $a$ (so $a$ can take values in the domain $[-1,1]$)*

**BE AWARE** 

In Notebooks: *03_Maximum_Likelihood_Amplitude_Estimation_Class.ipynb*, *04_Classical_Phase_Estimation_Class.ipynb*, *05_Iterative_Quantum_Phase_Estimation_Class.ipynb*, *06_Iterative_Quantum_Amplitude_Estimation_class.ipynb* we want to estimate $\sqrt{a}$ meanwhile in this new problem we want to estimate $a$


### 2.2 RQAE algorithm output

Given an error $\epsilon$ and a confident interval $\gamma$, the **RQAE** algorithm allows to estimate the $a$, from the amplitude estimation problem presented in *Section 2.1*, satisfying:

$$P\big[a \in [a_l, a_u]\big] \gt 1-\gamma$$

and

$$\frac{a_u-a_l}{2} \leq \epsilon$$


### 2.3 Creating object from the RQAE class

We have implemented a Python class called **RQAE** into the **QQuantLib/AE/real_quantum_ae** module that allows us to use the **RQAE** algorithm.

For creating the **RQAE** class the conventions used in **MLAE class** from **QQuantLib/AE/maximum_likelihood_ae.py** module should be followed: 

We have some mandatory inputs:

1. Oracle: QLM AbstractGate or QRoutine with the implementation of the Oracle for creating the Grover operator.
2. target: this is the marked state in binary representation as a Python list. 
3. index: list of the qubits affected by the Grover operator. 

And some optional inputs, used for algorithm configuration, that can be given as a Python dictionary:
* qpu: QLM solver that will be used
* epsilon ($\epsilon$): the precision. Ensures that the width of the interval is (see Section 2.2), at most, $2\epsilon$ (default: 0.01).
* gamma ($\gamma$): the accuracy (or final failure probability). Ensures that the probability of $a$ not laying within the given interval (see Section 2.2) is, at most, $\gamma$ (default: 0.05).
* ratio: the amplification ratio (default: 2).
* mcz_qlm: for using QLM multi-controlled Z gate (True, default) or using multiplexor implementation (False)

In [ ]:
#import the class
from QQuantLib.AE.real_quantum_ae import RQAE

To show how our **RQAE** class works, we will define the following amplitude estimation problem:

$$|\Psi\rangle = \mathcal{A}|0\rangle = \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right] \tag{2}$$

So comparing (2) with (1):

$$a|\Psi_0\rangle =  \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle$$

and 

$$\sqrt{1-a^2}|\Psi_1\rangle = \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

The target state, in this case, is $|1\rangle$. Its binary representation is $001$. This has to be passed to the target variable as a list. Moreover, we have to provide the list of qubits where we are acting, in this case is just $[0,1,2]$, the whole register.

In [ ]:
target = [0,0,1]
index = [0,1,2]

#We want to estimate the probability of target.
#In the RQAE solution the $a$ is codified as an amplitude not as a probability
a_real = np.sqrt(probability[bitfield_to_int(target)])

print('We want to estimate: ', a_real)

rqae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True   
}

rqae = RQAE(oracle,target = [0,0,1],index = [0,1,2], **rqae_dict)

### 2.4 The *rqae* method

To execute the complete algorithm using the **RQAE** class the *rqae* method can be used. 

 This method has the following inputs:
* ratio: the amplification ratio
* epsilon ($\epsilon$): error in the estimation of $a$ (default: 0.01).
* gamma ($\gamma$): confidence interval (failure probability) for the $a$ estimation (default: 0.05).

This method returns the limits for the $a$ estimation: $(a_{\min},a_{\max})$

In [ ]:
epsilon = 0.01
gamma = 0.05
q = 2.0
bounds = rqae.rqae(ratio=q, epsilon=epsilon, gamma=gamma)

In [ ]:
print('Bounds for a: [a_l, a_u] = [{}, {}]'.format(bounds[0], bounds[1]))
a_estimated = (bounds[0]+bounds[1])/2.0
print('a_estimated: ', a_estimated)
print('a real: ', a_real)
print('|a_l-a_estimated| = ', np.abs(a_estimated-a_real))
print('Error estimation wanted: ', 0.01)

In [ ]:
if (a_real>=bounds[0])&(a_real<=bounds[1]):
    print("Correct")
else:
    print("Incorrect")

Additionally, the **rqae** method populates the *time_pdf* property where several times for each iteration of the algorithm are stored. 

The **rqae_overheating** column in *time_pdf* property refers to pure **rqae** step algorithm times

In [ ]:
rqae.circuit_statistics

### 2.5 The *display_information* method

This method provides technical information  about the **RQAE** algorithm for a fixed configuration of:
* ratio ($q$ in the **RQAE** paper): amplification between steps
* epsilon ($\epsilon$): desired error in the estimation of $a$
* gamma ($\gamma$): confidence level (failure probability)

In [ ]:
rqae.display_information(ratio = q,epsilon = epsilon, gamma = gamma)

### 2.6 The *run* method

Finally, a *run* method for direct implementation of the **RQAE** algorithm was implemented. In this case, the user can configure all the properties of the **RQAR** class and the *run* method will execute the method using the fixed attributes of the class. Finally, the method returns the estimation of $a=\frac{a_u+a_l}{2}$. Additionally, the *run* method populates the following class attributes:

* *ae_l*: the lower limit for a $a_l$.
* *ae_u*: the upper limit for a $a_u$.
* *ae*: the amplitude estimation parameter calculated as: $a=\frac{a_u+a_l}{2}$
* *run_time*: elapsed time for a complete execution of the **run** method.


In [ ]:
#First we create the class
target = [0,0,1]
index = [0,1,2]
a = np.sqrt(probability[bitfield_to_int(target)])

print('We want to estimate: ', a)

epsilon = 0.001
q = 2
gamma = 0.01

rqae_dict = {
    'qpu': linalg_qpu,    
    'epsilon': epsilon,
    'ratio': q,
    'gamma': gamma,
    'mcz_qlm': False
}

rqae_ = RQAE(oracle, target = target, index = [0,1,2], **rqae_dict)

In [ ]:
a_estimated = rqae_.run()

In [ ]:
print('a_estimated: ', a_estimated)
print('Real Value of a: ', a_real)
print('Bounds for a: [rqae.a_l, rqae.a_u] = [{}, {}]'.format(rqae_.ae_l, rqae_.ae_u))
print('Estimated a: rqae.a= ', rqae_.ae)
print('|a_l-a_estimated| = ', np.abs(a-rqae_.ae))
print('Error estimation wanted: ', rqae_.epsilon)

In [ ]:
print("Elapsed time of a run method: ", rqae_.run_time)

In [ ]:
grover = rqae_._grover_oracle

%qatdisplay grover --depth 3 --svg

When the *run* method is executed the following class attributes are populated:

* *circuit_statistics*: Python dictionary with the statistics of each circuit used during the algorithm execution. Each key of the dictionary corresponds with a $k$ application of the Grover-like operator used and its associated value is a Python dictionary with the complete statistical information of the circuit created for each $k$ value.
* *schedule_pdf*: pandas DataFrame with the complete schedule used in the algorithm execution. The schedule lists the number of applications Grover-like applications and the number of shots used for measurements.
* *oracle_calls*: number of total oracle calls for a complete algorithm's execution.
* *max_oracle_depth*: maximum number of applications of the oracle for the complete algorithm's execution.

In [ ]:
rqae_.circuit_statistics

In [ ]:
rqae_.schedule_pdf

In [ ]:
#Total number of oracle calls
print("The total number of the oracle calls for the IQAE was: {}".format(rqae_.oracle_calls))

In [ ]:
#Number of maximum oracle applications
rqae_.max_oracle_depth

In [ ]:
rqae_.quantum_time

In [ ]:
rqae_.run_time

## 3. The RQAE Algorithm

In this section, we provide an insight into how the **RQAE** algorithm works. We are going to use the *oracle*, the *target* and the *index* from section 1. 

The **RQAE** algorithm has 2 well different parts:
* First step: where the sign of the amplitude $a$ can be estimated.
* The following interactions where the Grover operators are used for boosting the estimation.

You should remember that we have an operator $\mathcal{0}$ such that:

$$\mathcal{0}|0\rangle = |\Psi\rangle = a|\Psi_0\rangle +\sqrt{1-a^2}|\Psi_1\rangle, \tag{1}$$

with $a\in[-1, 1]$ and we want an estimation of $a$, $\hat{a}$

To explain the algorithm we are going to use the following distribution:

In [ ]:
from QQuantLib.utils.utils import bitfield

In [ ]:
n = 4
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)
# This will be the amplitude that we want to estimate
value = np.max(probability)
target_id = np.argmax(probability)
target = bitfield(target_id, n)
index = list(range(n))
print("Amplitude to estimate: ", str(value))
print("Integer for the Amplitude to estimate: ", str(target_id))
print("State with the desired Amplitude: ", str(target))

In [ ]:
# This is the A operator
oracle = load_probability(probability)

In [ ]:
%qatdisplay oracle --svg

In [ ]:
results,_,_,_ = get_results(oracle, linalg_qpu, shots=0)

In [ ]:
#Codified value in target_id vs value to estiamte
print(list(results[results["Int_lsb"] == target_id]["Probability"])[0], value)

Now we create the **RQAE** object

In [ ]:
epsilon = 0.001
q = 2
gamma = 0.01

rqae_dict = {
    'qpu': linalg_qpu,    
    'epsilon': epsilon,
    'ratio': q,
    'gamma': gamma,
    'mcz_qlm': False
}
rqae_obj = RQAE(oracle=oracle, target=target, index=index, **rqae_dict)

### 3.1 First step

The first iteration of the **RQAE** aims to get a first estimation of the amplitude and the corresponding sign. 

For doing this instead of using the $\mathcal{0}$ operator (the *oracle* object in the code) the **RQAE** will use a version of the operator $\mathcal{0}_b$ that depends on $\mathcal{0}$. The $b$ from the $\mathcal{0}_b$ is a number between $[-0.5, 0.5]$ that it is called the **shift**. Meanwhille the $\mathcal{0}$ acts upon $n$ qubits the $\mathcal{0}_b$ acts upon $n+1$ qutbits. The circuit implementation is shown in the following figure:

![title](images/rqae.svg)

This $\mathcal{0}_b$ is composed of 3 different operators (the three boxes in the diagram):
* $R_y(2 \theta_b)$: a rotation around axis y of $2 \theta_b$ with $\theta_b = \arccos(b)$. Main behaviour is: $$R_y(2\theta_b) \ket{0} = \cos(\theta_b) \ket{0} + \sin(\theta_b) \ket{1}$$
* The *Mask* operator. This operator is an operator whose main behaviour is the following:

$$Mask \ket{0}\otimes \ket{0}^n \rightarrow \ket{0} \otimes \ket{\Psi_0}$$
$$Mask \ket{0}\otimes \ket{i \ne 0}^n \rightarrow \text{any state that can not be} \ket{0} \otimes \ket{\Psi_0}$$
* The oracle operator $\mathcal{0}$
$$\mathcal{0}|0\rangle = |\Psi\rangle = a|\Psi_0\rangle +\sqrt{1-a^2}|\Psi_1\rangle$$


Now we are going to analyze the complete action of the $\mathcal{0}_b$ circuit:


1. $\ket{0}\otimes \ket{0}^n$
2. $(H \otimes I^n) \ket{0}\otimes \ket{0}^n = \frac{1}{\sqrt{2}} ( \ket{0}\otimes \ket{0}^n + \ket{1}\otimes \ket{0}^n)$
3. The first qubit controls a y-rotation gate over the last qubit of $\theta_b$:
$$\biggl(\ket{0}\bra{0} \otimes I ^{n-1} \otimes I  + \ket{1}\bra{1} \otimes I ^{n-1} \otimes R_y(2\theta_b) \biggl) \frac{1}{\sqrt{2}} \big( \ket{0} +\ket{1} \big) \otimes \ket{0}^{n-1} \otimes \ket{0}=$$
$$=\frac{1}{\sqrt{2}} \biggl(\ket{0} \otimes \ket{0}^{n} +\cos(\theta_b) \ket{1} \otimes \ket{0}^{n} + \sin(\theta_b) \ket{1} \otimes \ket{0}^{n-1} \otimes \ket{1} \biggl)=$$
$$=\frac{1}{\sqrt{2}} \biggl(\ket{0} \otimes \ket{0}^{n} +\cos(\theta_b) \ket{1} \otimes \ket{0}^{n} + \cdots \biggl)$$    
4. The first qubit controls the mask operator over the other $n$ qubits. Using the behaviour of the Mask operator we arrive to:

$$ \biggl(\ket{0}\bra{0} \otimes I ^{n} + \ket{1}\bra{1} Mask \biggl) \frac{1}{\sqrt{2}} \biggl(\ket{0} \otimes \ket{0}^{n} +\cos(\theta_b) \ket{1} \otimes \ket{0}^{n} + \cdots \biggl)=$$ 
$$ = \frac{1}{\sqrt{2}} \biggl(\ket{0} \otimes \ket{0}^{n} + \cos(\theta_b) \ket{1} \otimes \ket{\Psi_0} + \cdots \biggl)$$
5. The first qubit anti-control the oracle operator $\mathcal{0}$ over the last $n$ qbits:

$$\biggl(\ket{0}\bra{0} \otimes \mathcal{0}  + \ket{1}\bra{1} \otimes  I ^{n}  \biggl) \frac{1}{\sqrt{2}} \biggl(\ket{0} \otimes \ket{0}^{n} + \cos(\theta_b) \ket{1} \otimes \ket{\Psi_0} + \cdots \biggl)=$$

$$ = \frac{1}{\sqrt{2}} \biggl( a \ket{0} \otimes \ket{\Psi_0} + \cos(\theta_b) \ket{1} \otimes \ket{\Psi_0} + \cdots \biggl)$$
6. Now a Haddamard gate is applied over the first qubit:

$$ (H \otimes I^n) \frac{1}{\sqrt{2}} \biggl( a \ket{0} \otimes \ket{\Psi_0} + \cos(\theta_b) \ket{1} \otimes \ket{\Psi_0} + \cdots \biggl) = $$

$$=\frac{1}{2} \biggl( \big(a + \cos(\theta_b)\big) \ket{0} \otimes \ket{\Psi_0} +  \big(a - \cos(\theta_b)\big) \ket{1} \otimes \ket{\Psi_0} + \cdots \biggl) $$


So the main action of the $\mathcal{0}_b$ operator is:

$$\mathcal{0}_b \ket{0} \otimes \ket{0}^n = \left(\frac{\cos \theta_b + a}{2} \right)  \ket{0} \otimes \ket{\Psi_0} + \left(\frac{\cos \theta_b - a}{2} \right)  \ket{1} \otimes \ket{\Psi_0} + \cdots$$


So the probability of getting the $\ket{0} \otimes \ket{\Psi_0}$ state is given by:

$$P_{\ket{0} \otimes \ket{\Psi_0}} = \left(\frac{\cos \theta_b + a}{2} \right) ^2$$

and the probability of getting the $\ket{1} \otimes \ket{\Psi_0}$ state is given by:

$$P_{\ket{1} \otimes \ket{\Psi_0}} = \left(\frac{\cos \theta_b - a}{2} \right) ^2$$

So in the first step the algorithms creates the $\mathcal{0}_b$ and measures the states $\ket{0} \otimes \ket{\Psi_0}$ and  $\ket{1} \otimes \ket{\Psi_0}$  and a first estimation of $a$, $\hat{a}^{1st}$ is obtained by: 

$$\hat{a}^{1st} = \frac{P_{\ket{0} \otimes \ket{\Psi_0}} - P_{\ket{1} \otimes \ket{\Psi_0}}}{\cos \theta_b}$$

In this iteration the sign of $a$ can be recovered (if $P_{\ket{1} \otimes \ket{\Psi_0}} > P_{\ket{0} \otimes \ket{\Psi_0}}$ then $\hat{a}^{1st} < 0$).

Once the $\hat{a}^{1st}$ is estimated the corresponding bounds, $[\hat{a}^{1st}_L, \hat{a}^{1st}_U]$  can be obtained by using the *Chebyshev* inequality. For this computation the desired failure probability, $\gamma_i$ and the number of shots used for obtaining the $\hat{a}^{1st}$ must be provided.

The $\mathcal{0}_b$, also called the *shifted oracle*, can be constructed by assigning to the *shifted_oracle* attribute a desired shift ($b$). For obtaining the corresponding $\mathcal{0}_b$ we can use the attribute *_shifted_oracle*

In [ ]:
# this is the input oracle
c = rqae_obj.oracle
%qatdisplay c --svg
# Now we create the shifted oracle:
shift = 0.5 
rqae_obj.shifted_oracle = shift

In [ ]:
# The corresponding shifted oracle circuit
c = rqae_obj._shifted_oracle
%qatdisplay c --svg

The method *first_step* allows the user to execute the first step of the algorithm by providing:
* shift: the $b$ for creating the *shifted oracle*
* shots: number of shots for doing the measures
* gamma: desired failure probability for computing the bounds $[\hat{a}^{1st}_L, \hat{a}^{1st}_U]$

This method returns directly the bounds:  $[\hat{a}^{1st}_L, \hat{a}^{1st}_U]$ and the circuit used.

In [ ]:
shift = 0.2
shots = 100
gamma = 0.01
[a_first_low, a_first_upper], circuit = rqae_obj.first_step(shift=shift, shots=shots, gamma=gamma)

In [ ]:
print("a_first_low: ", a_first_low)
print("a_first_upper: ", a_first_upper)    

In [ ]:
%qatdisplay circuit --svg

**NOTE**

The **RQAE** algorithm sets the shift, the number of shots and the desired failure probability for the first step automatically to obtain a performance that can be compatible with other state-of-art Amplitude Estimation algorithms (like Grinko's **IQAE**)

### 3.2 Following iterations

Once the first estimation, $[\hat{a}^{1st}_L, \hat{a}^{1st}_U]$, is obtained then the following iterations try to reduce this interval by exploiting the amplification capabilities of the *Grover* operator. 

For a step $t$ of the iterative process, first a *shited oracle* is created using as shift the lower bound of the last step $b=a^{t-1}_L$, using this *shited oracle* operator, $\mathcal{O}_{b=a^{t-1}_L}$ the corresponding Grover operator $\mathcal{G}(\mathcal{O}_{b=a^{t-1}_L})$ is created. Then the following circuit is executed:

$$\mathcal{G}^k (\mathcal{O}_{b=a^{t-1}_L})\mathcal{O}_{b=a^{t-1}_L} \ket{0}^n \otimes \ket{0}$$

Where $k$ is the number of times the Grover operator should be applied (depends on the step $t$ of the algorithm). 

Then the probability of the state $\ket{0} \otimes \ket{\Psi_0}$ should be measured. The estimation of the step $t$ is then: $\hat{a}^t = P_{\ket{0} \otimes \ket{\Psi_0}}$

Again using the *Chebyshev* inequality the corresponding bounds,  $[\hat{a}^{t}_L, \hat{a}^{t}_U]$, can be obtained.

A $t$ step is executed using the *run_step* method. The inputs are:
* shift
* shots
* gamma: desired step failure probability
* k: amplification

Again the *run_step* method returns the bounds:  $[\hat{a}^{t}_L, \hat{a}^{t}_U]$ and the circuit used.

In [ ]:
[a_t_low, a_t_upper], circuit = rqae_obj.run_step(a_first_low, 100, 0.02, 4)

In [ ]:
print("a_t_low: ", a_t_low)
print("a_t_upper: ", a_t_upper)  

In [ ]:
%qatdisplay circuit --svg 

**NOTE**

The **RQAE** algorithm sets the shift, the number of shots, the desired failure probability and the amplification $k$ for each step automatically to obtain a performance that can be compatible with other state-of-art Amplitude Estimation algorithms (like Grinko's **IQAE**=